# NKSOPS_02_ClassAndMethods_01

In [4]:
#load Enviornment
%run NKSOPS_01_Environment_01.ipynb

In [5]:
# Class and Methods

class NKS_OPS:
    
    def __init__(self, input_path, input_columns, export_parent_dir, export_columns, research_topic
                 , severity, logfile, sqlite_create):
        self.input_path = input_path
        self.input_columns = input_columns
        self.export_parent_dir = export_parent_dir
        self.export_columns = export_columns
        self.research_topic = research_topic
        self.severity = severity
        self.logfile = logfile
        self.sqlite_create = sqlite_create
        
    # Create the research directory for the given severity of the study
    def create_research_directory(self):
        name = self.research_topic + "_" + self.severity
        export_path = os.path.join(self.export_parent_dir, name)
        if not os.path.exists(export_path):
            export_path = os.mkdir(export_path)
        return export_path
    
    # Transform
    def data_transformation(self, filename):
        df = pd.read_csv(filename, usecols= self.input_columns, low_memory=False)
#         df.columns = [col.strip() for col in df.columns]
        return df
    
    # NKS_OPS_Data_Study
    def NKS_OPS_Data_Study(self):
         # Read Each File and do
        for filename in glob.glob(self.input_path):

            # Create the research directory for the given severity of the study
            self.create_research_directory()

            # Generate path for exporting csv
            base_filename = os.path.basename(filename)
            export = os.path.join(self.create_research_directory(), base_filename)

            # Read and do inital Transformation of csv file
            df = self.data_transformation(filename = filename)

            # Export CSV file for study
            df.rename_axis('ID').to_csv(export, header='column_names', columns= self.export_columns)
    
    # Append to single dataframe
    def Append_to_DF(self):
        path = self.export_parent_dir
        all_files = glob.glob(path  + "/" + self.research_topic + "_" + self.severity + "/*.csv")
        li = []
        for filename in all_files:
            df = pd.read_csv(filename, index_col=None, header=0)
            base_filename = os.path.basename(filename)
            df.loc[:, 'Unique_ID'] = base_filename.replace(".csv","") + '_' + self.research_topic + '_' + self.severity
            df.loc[:, 'FILE_PATH'] = filename
            li.append(df)
        frame = pd.concat(li, axis=0, ignore_index=True)   
        return frame
    
    # Extract text beteeen two points
    def Extract_Between_Text(self, df, newcolumn, textcolumn, start, end):
        df.loc[:,newcolumn] = df.loc[:,textcolumn].astype(np.object).str.split(pat=start).str[1]
        df.loc[:,newcolumn] = df.loc[:,newcolumn].astype(np.object).str.split(pat=end).str[0]
        return df.loc[:,newcolumn]
    
    def create_connection(self,db_file):
        """ create a database connection to the SQLite database
            specified by db_file
        :param db_file: database file
        :return: Connection object or None
        """
        conn = None
        try:
            conn = sqlite3.connect(db_file)
            return conn
        except Error as e:
            print(e)

        return conn
 
 
    def create_table(self,conn, create_table_sql):
        """ create a table from the create_table_sql statement
        :param conn: Connection object
        :param create_table_sql: a CREATE TABLE statement
        :return:
        """
        try:
            c = conn.cursor()
            c.execute(create_table_sql)
        except Error as e:
            print(e)
 
 
    def NKS_OPS_DB(self,db_file):
        database = db_file

        NKS_OPS_TABLE = """ CREATE TABLE IF NOT EXISTS NKS_OPS (
                                            Unique_ID text PRIMARY KEY,
                                            Received text,
                                            Contents text,
                                            MESSAGETYPE text,
                                            DATE text,
                                            TAIL text,
                                            STATION text,
                                            DESCRIPTION text,
                                            TROUBLESHOOTING text,
                                            CORRECTIVEACTION text,
                                            STD_GMT text,
                                            ETRS_DATE text,
                                            RTS_DATE text,
                                            DESTINATION text,
                                            Unique_Flight_Deprt text,
                                            Unique_Flight_Arrvl text,
                                            Event_Order text
                                        ); """
        
        NKS_FLIGHTS_TABLE = """ CREATE TABLE IF NOT EXISTS NKS_FLIGHTS (
                                            Unique_Flight text PRIMARY KEY,
                                            ID integer,
                                            Unique_Flight_Deprt text,
                                            Unique_Flight_Arrvl text,
                                            Actual_Delay text,
                                            Delay_Code text,
                                            Depart_Date text,
                                            Flight text,
                                            Apt_Deprt text,
                                            Apt_Arrvl text,
                                            Tail text,
                                            Depart_Day text,
                                            Depart_Month text,
                                            Depart_Year text,
                                            FILE_PATH text
                                        ); """

        # create a database connection
        conn = self.create_connection(database)

        # create tables
        if conn is not None:
            # create projects table
            self.create_table(conn, NKS_OPS_TABLE)
            conn.close()
        else:
            print("Error! cannot create the database connection.")
            conn.close()
    